In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [31]:
sector_file = "Support_Data/constituents.csv"
sector_df = pd.read_csv(sector_file)
sector_df.columns = ['symbol','name','sector']
sector_df.head()

,symbol,name,sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [32]:
SP500Landing_file = "Support_Data/SP_500_Sectors.csv"
SP500_df = pd.read_csv(SP500Landing_file)
SP500_df.head()

,Ticker,Name,Price,Dividend Yield,Market Cap ($M),P/E Ratio,Payout Ratio,Beta
0,A,"Agilent Technologies, Inc.",76.64,0.8,"23,717",22.7,19,1.11
1,AAL,"American Airlines Group, Inc.",30.76,1.3,"13,474",8.7,11.3,1.45
2,AAP,"Advance Auto Parts, Inc.",167.4,0.1,"11,950",27.8,4,0.67
3,AAPL,"Apple, Inc.",260.14,1.2,"1,155,872",21.8,25.1,1.54
4,ABBV,"AbbVie, Inc.",85.21,5.0,"125,981",39.1,196.3,0.86


In [33]:
SP500_cols = ["Ticker", "Price", "P/E Ratio", "Beta"]
SP500_transformed = SP500_df[SP500_cols].copy()

SP500_transformed = SP500_transformed.rename(columns={"Ticker": "Symbol", "P/E Ratio": "PE_Ratio"})
SP500_transformed.head()

,Symbol,Price,PE_Ratio,Beta
0,A,76.64,22.7,1.11
1,AAL,30.76,8.7,1.45
2,AAP,167.4,27.8,0.67
3,AAPL,260.14,21.8,1.54
4,ABBV,85.21,39.1,0.86


In [34]:
SP500_transformed.columns = ['symbol','price','pe_ratio','beta']

In [36]:
sp500_transformed = SP500_transformed
sp500_transformed.head()

,symbol,price,pe_ratio,beta
0,A,76.64,22.7,1.11
1,AAL,30.76,8.7,1.45
2,AAP,167.4,27.8,0.67
3,AAPL,260.14,21.8,1.54
4,ABBV,85.21,39.1,0.86


In [49]:
sp500_transformed['price'] = sp500_transformed['price'].str.replace(',','')
sp500_transformed['pe_ratio'] = sp500_transformed['pe_ratio'].str.replace(',','')


In [50]:
# SP500_transformed['price'] = SP500_transformed['price'].replace(',','')
# sp500_transformed['price'] = sp500_transformed['price']

my_dict = {'symbol':'str'
          ,'price':'float64'
          ,'pe_ratio':'float64'
          ,'beta':'float64'}
sp500_transformed = sp500_transformed.astype(my_dict)
sp500_transformed.dtypes

symbol       object
price       float64
pe_ratio    float64
beta        float64
dtype: object

In [51]:
connection_string = "postgres:postgres@localhost:5432/ETL_Project_Chen_Stewart"
engine2 = create_engine(f'postgresql://{connection_string}')

In [52]:
engine2.table_names()

['table_one', 'table_two']

In [53]:
sector_df.to_sql(name="table_one", con=engine2, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "table_one_pkey"
DETAIL:  Key (symbol)=(MMM) already exists.

[SQL: INSERT INTO table_one (symbol, name, sector) VALUES (%(symbol)s, %(name)s, %(sector)s)]
[parameters: ({'symbol': 'MMM', 'name': '3M Company', 'sector': 'Industrials'}, {'symbol': 'AOS', 'name': 'A.O. Smith Corp', 'sector': 'Industrials'}, {'symbol': 'ABT', 'name': 'Abbott Laboratories', 'sector': 'Health Care'}, {'symbol': 'ABBV', 'name': 'AbbVie Inc.', 'sector': 'Health Care'}, {'symbol': 'ACN', 'name': 'Accenture plc', 'sector': 'Information Technology'}, {'symbol': 'ATVI', 'name': 'Activision Blizzard', 'sector': 'Information Technology'}, {'symbol': 'AYI', 'name': 'Acuity Brands Inc', 'sector': 'Industrials'}, {'symbol': 'ADBE', 'name': 'Adobe Systems Inc', 'sector': 'Information Technology'}  ... displaying 10 of 505 total bound parameter sets ...  {'symbol': 'ZION', 'name': 'Zions Bancorp', 'sector': 'Financials'}, {'

In [55]:
sp500_transformed.to_sql(name="table_two", con=engine2, if_exists='append', index=False)